<a href="https://colab.research.google.com/github/sravan1320/NMT/blob/main/fine_tune_hugging_face_translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### If you dont want to use Wandb, disable Wandb otherwise optional

references for WANDB
https://analyticsindiamag.com/hands-on-guide-to-weights-and-biases-wandb-with-python-implementation/

https://docs.wandb.ai/


In [121]:
import os
#os.environ["WANDB_DISABLED"]="true"

In [122]:
#!conda install -c conda-forge ipywidgets

# Install required packages

In [123]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version

In [124]:
import transformers
print(transformers.__version__)
import torch
print(torch.cuda.is_available())

4.15.0
False


# Fine-tuning a model on a translation task

In [125]:
model_checkpoint = "Helsinki-NLP/opus-mt-mul-en"

### Loading the dataset

We will use the [datasets](https://github.com/huggingface/datasets/tree/master/datasets/wmt16) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions load_dataset and load_metric. We use the English/Romanian part of the WMT dataset here

In [126]:
from datasets import load_dataset, load_metric
metric = load_metric("sacrebleu")

The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

You can call its compute method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

# Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

If you downloaded the model manually, you can provide model present directory instead of model_checkpoint

In [127]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/resolve/main/config.json from cache at C:\Users\Alienware/.cache\huggingface\transformers\07abbdf03be1b9f5bb00584b076297530595fae883f8715a86b8ee70ba288ad9.209344e751b882bdde512ed43f02eefed0bd84b70b20c62d12070906bfff04a3
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-mul-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      64171
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 64171,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_to

You can directly call this tokenizer on one sentence or a pair of sentences:

In [128]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[12899, 3, 73, 151, 18617, 58, 0], [302, 17, 675, 18617, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [129]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 88EC-B3A6

 Directory of C:\Users\Alienware\Desktop

01/17/2022  07:22 PM    <DIR>          .
01/17/2022  07:22 PM    <DIR>          ..
01/17/2022  12:54 AM    <DIR>          .ipynb_checkpoints
12/29/2021  05:58 PM             2,702 æTorrent.lnk
12/11/2021  01:56 PM           105,288 16523926_980916128706235_1051899573_o.png
12/16/2021  10:36 PM           141,442 64fa674b-d341-46ff-9a02-faba283bd961.pdf
01/17/2022  01:02 AM    <DIR>          archive
01/17/2022  01:02 AM        39,340,674 archive.zip
12/13/2021  12:10 PM           982,120 ASEPTCONN-AG-sonea (1).pdf
01/07/2022  09:48 AM           982,120 ASEPTCONN-AG-sonea (2).pdf
01/11/2022  12:05 PM           982,120 ASEPTCONN-AG-sonea.pdf
11/29/2021  11:47 AM       147,252,696 autocad_plant_3d_2022_sdk_english_win_64bit_dlm.sfx.exe
01/13/2022  11:34 AM         1,123,667 bilet.pdf
01/17/2022  10:41 AM       653,088,243 CCdf.csv
01/17/2022  01:00 AM    <DIR>          CCMatrix
01/

conda install -c anaconda scikit-learnWe can then write the function that will preprocess our samples. We just feed them to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [130]:
conda install -c anaconda scikit-learn


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\Alienware\miniconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [131]:
import pandas as pd
import sklearn

In [132]:
train_df = pd.read_csv('archive/train_df.csv')
test_df = pd.read_csv('archive/test_df.csv')
train_df['ro'] = train_df.ro.astype(str).apply(lambda x: x.replace('</i>', ''))
test_df['ro'] = test_df.ro.astype(str).apply(lambda x: x.replace('</i>', ''))
train_df['fa'] = train_df.fa.astype(str).apply(lambda x: x.replace('</i>', ''))
test_df['fa'] = test_df.fa.astype(str).apply(lambda x: x.replace('</i>', ''))

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=len(train_df)//100, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

import datasets
from datasets import Dataset, dataset_dict

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

raw_datasets = dataset_dict.DatasetDict({'train':train_dataset, 'validation':val_dataset, 'test':test_dataset})

In [133]:
cc_matrix_df

,Unnamed: 0,ro,fa,source
0,0,"Aceasta este viata pe care o doresc! "", Atunci...",این زندگی است که من می خواهم! »سپس گام بعدی سا...,CCMatrix
1,1,Irmosul: Pe Dumnezeu Cuvântul cel din Dumnezeu...,اصلاً ابلیس یک-موجود- مکّاری است؛ به خدا قسم!,CCMatrix
2,2,Fresno Pacific University - este posibil să se...,Fresno Pacific University - ممکن است در اینجا ...,CCMatrix
3,3,De ce vă amestecaţi într-o plăcere care.,۱ ۱ Im(,CCMatrix
4,4,De ce vă amestecaţi într-o plăcere care.,شرح حکایت 1 (,CCMatrix
...,...,...,...,...
2694925,2694925,Însănătoșirea lumii,بهبود سلامت جهان,CCMatrix
2694926,2694926,"Dacă luăm în considerare doar ulterior, Red Mc...",اما اگر ما تنها حیات وحش مککام در تگزاس را با ...,CCMatrix
2694927,2694927,În Italia sunt din belşug.,من از ایتالیا ناخشنودم.,CCMatrix
2694928,2694928,Parteneriatul public privat reprezintă o soluţie.,جنگ بشردوستانه راه حل است.,CCMatrix


In [134]:
train_df

,Unnamed: 0,ro,fa,source
0,61903,"Nu-mi place de loc neutralitatea în viaţă, în ...",من خنثی بودن را نه در زندگی و نه در هیچ چیز دی...,TED2020
1,53955,Am fost ales să construiesc un pavilion din tu...,بنابراین از من خواسته شد که که غرفه‌ای از لوله...,TED2020
2,290703,când ne socoteam deopotrivă cu Domnul lumilor!,چرا که شما را با پروردگار جهانیان برابر می‌شمر...,Tanzil
3,91500,De ce? Am scris o carte în care am încercat să...,چرا؟ من یک کتاب کامل نوشته‌ام تا این موضوع را ...,TED2020
4,266235,"În ultimul deceniu, șapte milioane de străini ...",در طی دهه گذشته، هفت میلیون خارجی شهروندی آمری...,TED2020
...,...,...,...,...
348879,119879,"Femeile si copii, in special cei saraci, sunt ...",بچه ها و زنان، بخصوص فقرا، در قسمت پایین آن نر...,TED2020
348880,259178,"Dacă veţi număra binefacerile lui Dumnezeu, nu...",(خداوند جز اینها دارای نعمتهای فراوان دیگری اس...,Tanzil
348881,131932,În Ziua când pământul se va despica deasupra l...,روزى كه زمين به سرعت از [اجساد] آنان جدا و شكا...,Tanzil
348882,146867,Și adevărul este că nu înțeleg pe deplin tot c...,و حقیقت اینست که من در واقع بسیاری از چیزهاییک...,TED2020


In [135]:
#train_df = pd.concat(train_df, cc_matrix_df.iloc[:90000]).reset_index(drop=True))
cc_matrix_df = pd.read_csv('CCdf.csv')
train_df = pd.concat([train_df, cc_matrix_df.iloc[:99900]]).reset_index(drop=True)

In [136]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "fa"
target_lang = "ro"
def preprocess_function(dataset):
    inputs = dataset[source_lang]
    targets = dataset[target_lang]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [137]:
preprocess_function(train_dataset[:2])

{'input_ids': [[856, 2471, 176, 1860, 196, 6138, 176, 700, 910, 1164, 31228, 152, 910, 1164, 8595, 15634, 22522, 20870, 5520, 926, 259, 2, 0], [11976, 2958, 1613, 176, 1682, 856, 1635, 7255, 6836, 507, 507, 5280, 180, 17870, 26, 1613, 1682, 14321, 4184, 16823, 664, 723, 5174, 10297, 583, 19444, 259, 118, 2134, 1841, 3334, 7040, 21276, 2574, 5306, 11475, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[1782, 20, 198, 731, 43, 300, 126, 22679, 721, 44754, 4, 1864, 71, 2199, 13642, 2671, 3, 4, 1864, 71, 97, 9026, 2, 0], [2694, 1016, 256, 9, 1884, 181, 2671, 484, 6, 9607, 8217, 126, 227, 47568, 224, 157, 37455, 2282, 43, 627, 1320, 94, 3409, 29864, 1429, 106, 31356, 2671, 2, 0]]}

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command

In [138]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/349 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [139]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'fa', 'input_ids', 'labels', 'ro', 'source'],
        num_rows: 348884
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'fa', 'input_ids', 'labels', 'ro', 'source'],
        num_rows: 3524
    })
    test: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'fa', 'input_ids', 'labels', 'ro', 'source'],
        num_rows: 3300
    })
})

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [140]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/resolve/main/config.json from cache at C:\Users\Alienware/.cache\huggingface\transformers\07abbdf03be1b9f5bb00584b076297530595fae883f8715a86b8ee70ba288ad9.209344e751b882bdde512ed43f02eefed0bd84b70b20c62d12070906bfff04a3
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-mul-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      64171
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 64171,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_to

To instantiate a Seq2SeqTrainer, we will need to define three more things. The most important is the [Seq2SeqTrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [141]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    gradient_accumulation_steps=5,
#     fp16=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the cell and customize the weight decay. Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the predict_with_generate option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Model will save under **{model_name}-finetuned-{source_lang}-to-{target_lang}** directory

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [142]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [143]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [144]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the train method:

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: Unnamed: 0, ro, source, fa.
***** Running training *****
  Num examples = 348884
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 40
  Gradient Accumulation steps = 5
  Total optimization steps = 52332


Epoch,Training Loss,Validation Loss


Saving model checkpoint to opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-500
Configuration saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-500\config.json
Model weights saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-500\pytorch_model.bin
tokenizer config file saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-500\tokenizer_config.json
Special tokens file saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-500\special_tokens_map.json
Saving model checkpoint to opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-1000
Configuration saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-1000\config.json
Model weights saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-1000\tokenizer_config.json
Special tokens file saved in opus-mt-mul-en-finetuned-fa-to-ro\checkpoint-1000\special_tokens_map.json


In [ ]:
import os
for dirname, _, filenames in os.walk('opus-mt-mul-en-finetuned-fa-to-ro'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Our fine tuned model already saved under *opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000*

Load the model and translate some text from english to romanian

In [ ]:
# from transformers import MarianMTModel, MarianTokenizer
# src_text = ['My name is Sarah and I live in London']

# model_name = 'opus-mt-mul-en-finetuned-fa-to-ro/checkpoint-9000'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# print(tokenizer.supported_language_codes)


In [ ]:
# src_text = raw_datasets['validation'][:5]['fa']

In [ ]:
# model = MarianMTModel.from_pretrained(model_name)
# translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

In [ ]:
# [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [ ]:
# raw_datasets['validation'][:5]['ro']